# git reset

## 개요
`git reset`은 **Repository, Stage, Working Tree의 상태를 특정 commit 시점으로 복원**하는 명령어dla

```bash
# 기본 사용법
git reset [mode] [복원할_commit_id]
```

**핵심 특징**:
- **Commit history의 일부를 삭제**함 (히스토리 변경)
- 과거 특정 시점으로 되돌아감
- **Remote repository 공동작업 시에는 사용 금지!**

> **중요 원칙**  
> "나만 알고 있는 역사(Local)는 맘대로 고쳐도 되지만,  
> 남들도 알고 있는 역사(Remote)는 고치면 안 된다."
>
> - Push 전: `git reset` 자유롭게 사용 가능  
> - Push 후: `git revert` 사용 필수

## 3가지 Mode

### 비교표

| Mode | Repository | Stage | Working Tree | 주요 용도 |
|------|------------|-------|--------------|-----------|
| `--soft` | 복원됨 | 변경사항 유지 (staged) | 유지 | 여러 commit을 하나로 합침 (Squash) |
| `--mixed` (기본) | 복원됨 | 복원됨 | 유지 | commit 취소 후 수정하여 재commit |
| `--hard` | 복원됨 | 복원됨 | 복원됨 | 완전히 과거로 되돌림 (⚠️ 주의) |

### 1. `--soft` 모드
**"과거로 돌아가되, 그 이후 작업을 staged 상태로 보존"**

```bash
git reset --soft <commit_id>
```

**동작**:
- Repository를 지정한 commit으로 복원
- 취소된 commit들의 **모든 변경사항이 Stage에 보존됨** (바로 commit 가능)
- Working Tree는 그대로 유지

**사용 예시**:
```bash
# 최근 3개 commit을 하나로 합치기 (Squash)
git reset --soft HEAD~3
git commit -m "Combined 3 commits into one"
```

### 2. `--mixed` 모드 (기본값)
**"commit 취소 후 수정하여 다시 commit 하고 싶을 때"**

```bash
git reset <commit_id>
# 또는
git reset --mixed <commit_id>
```

**동작**:
- Repository와 Stage를 지정한 commit으로 복원
- 취소된 변경사항은 **Working Tree에만 남음** (unstaged)
- 파일을 수정하거나 나눠서 다시 commit 가능

**사용 예시**:
```bash
# 마지막 commit 취소 (변경사항은 Working Tree에 보존)
git reset HEAD^

# A, B, C 파일을 한 번에 commit 했는데, 나눠서 다시 commit
git reset HEAD^
git add A
git commit -m "Add file A"
git add B C
git commit -m "Add files B and C"
```

### 3. `--hard` 모드
**"완전히 과거 시점으로 되돌림 (모든 변경 삭제)"**

```bash
git reset --hard <commit_id>
```

**동작**:
- Repository, Stage, Working Tree **모두 지정한 commit 시점으로 복원**
- **모든 변경사항이 완전히 삭제됨** ⚠️

**⚠️ 위험**:
- Working Tree의 모든 변경사항이 삭제됨
- **복구 불가능**하므로 신중하게 사용

**사용 예시**:
```bash
# 지정한 commit으로 완전히 되돌림
git reset --hard 3fd92e

# 최근 2개 commit 완전 삭제
git reset --hard HEAD~2
```

## 상대 참조 (Relative Reference)

### HEAD를 기준으로 한 참조

```bash
# HEAD^ : HEAD의 바로 이전 commit (parent)
git reset HEAD^

# HEAD~n : HEAD로부터 n단계 이전 commit
git reset HEAD~1  # HEAD^ 와 동일
git reset HEAD~3  # 3단계 이전

# 차이점
HEAD^   # 첫 번째 parent (merge 시 직계 branch)
HEAD^2  # 두 번째 parent (merge 시 합쳐진 branch)
HEAD^^  # parent의 parent
```

## 실전 예제

### 예제 1: 최근 commit 취소 (변경사항 유지)
```bash
# 가장 최근 commit을 취소하고 Working Tree에 변경사항 남김
git reset HEAD^
# 또는
git reset HEAD~1
```

### 예제 2: 여러 commit을 하나로 합치기
```bash
# 최근 3개 commit을 하나로 합침
git reset --soft HEAD~3
git commit -m "Combine last 3 commits"
```

### 예제 3: 특정 commit으로 완전히 되돌리기
```bash
# 특정 commit 이후의 모든 것을 삭제
git reset --hard 3fd92e
```

### 예제 4: Stage에서 파일 제거 (옛날 방식)
```bash
# git restore --staged 가 없던 시절의 방식
git reset HEAD file.txt
```

## Remote Repository와 강제 Push

### ⚠️ 절대 권장하지 않음 (팀 작업 시 매우 위험)

만약 이미 push된 commit을 reset으로 삭제했다면:

```bash
# 1. Local에서 reset
git reset --hard HEAD~2

# 2. 강제 push (매우 위험!)
git push origin main --force

# 더 안전한 방법 (다른 사용자가 업데이트하지 않았을 때만 push)
git push origin main --force-with-lease
```

**강제 push를 하지 않으면**:
```bash
git push
# 거절됨!
# ! [rejected] main -> main (non-fast-forward)
# error: Updates were rejected because the tip of your current branch is behind
```

**왜 위험한가?**:
- 다른 개발자의 local repository와 history가 달라짐
- 다른 개발자가 pull 시 충돌 발생
- 팀 전체의 작업에 큰 혼란 초래

**대신 사용해야 할 방법**:
```bash
# Remote에 push된 commit은 revert로 취소
git revert <commit_id>
git push origin main  # 안전하게 push
```

## git reset vs git revert vs git restore

| 명령어 | 대상 시점 | 히스토리 | 협업 사용 | 주요 용도 |
|--------|-----------|----------|-----------|-----------|
| `git reset` | Commit 이후 | 삭제됨 | ⚠️ Local만 | Local의 commit 조작 |
| `git revert` | Commit 이후 | 보존됨 | ✅ 안전 | Remote commit 취소 |
| `git restore` | Commit 이전 | 변경 없음 | ✅ 안전 | Working Tree/Stage 되돌리기 |

## 작업 흐름 예시

```bash
# 1. Log 확인
git log --oneline

# 2. 되돌릴 commit 확인
# 예: abc1234 (3개 전) 시점으로 되돌리고 싶음

# 3-1. Local만 작업 중이면 (push 안 함)
git reset --hard HEAD~3

# 3-2. 이미 push 했으면
git revert HEAD~2..HEAD  # revert 사용!

# 4. 결과 확인
git log --oneline
```

## 주의사항 요약

⚠️ **절대 금지**
- Remote에 push된 commit을 `git reset`으로 삭제하는 것
- 팀원들과 상의 없이 강제 push (`--force`)

✅ **안전한 사용**
- Local에서만 작업 중일 때
- Push 하기 전에 commit을 정리할 때

⚠️ **`--hard` 주의**
- Working Tree의 모든 변경사항이 삭제됨
- **복구 불가능**

🔄 **협업 시 대안**
- Remote commit 취소: `git revert` 사용
- Working Tree 되돌리기: `git restore` 사용

---
**출처**: [ds31x.tistory.com/289](https://ds31x.tistory.com/289)
